# Análisis de resultados de varias carteras y posibilidad de representación gráfica

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [60]:
df = pd.read_csv("../tablas/dataformodel.csv", usecols=['Price_d', 'Price_d+180',
                                                        'quantile_PER', 'var_quantile_PER','PER','var_PER',
                                                        'quantile_PBC', 'var_quantile_PBC','var_PBC',
                                                        'quantile_ROA', 'var_quantile_ROA','var_ROA','ROA',
                                                        'quantile_Dot_dudosos', 'var_quantile_Dot_dudosos',
                                                        'Dot_dudosos', 'var_Dot_dudosos',
                                                        'Etiqueta', 'Periodo','Ticker'])

df=df.replace([np.inf, -np.inf], np.nan)
for column in df.columns:
    df=df[df[column].notnull()]
df=df.reset_index(drop=True)

In [16]:
def resultado(row):
    if row['Prediction'] == 'Peor':
        return (row['Price_d'] - row['Price_d+180'])/row['Price_d']
    else:
        return (row['Price_d+180'] - row['Price_d'])/row['Price_d']

In [67]:
lista_periodos=df.Periodo.unique()[20:-1]
titulos = 7
inversion = 100
resumen = pd.DataFrame(columns=['Periodo','Rentabilidad'])
ratios = ['quantile_PER','var_quantile_PER','quantile_PBC','var_quantile_PBC']

for trim_seleccionado in lista_periodos:
    df_train = df[df.Periodo<trim_seleccionado]
    df_test = df[df.Periodo==trim_seleccionado]
    
    X_train=df_train[ratios].values
    
    y_train=df_train['Etiqueta'].values
    
    
    X_test=df_test[ratios].values
    
    y_test=df_test['Etiqueta'].values
    
    clf=LogisticRegression(max_iter=200)
    clf.fit(X_train,y_train)
      
    df_prob=pd.DataFrame({'Probabilidad':clf.predict_proba(X_test)[:,0],'Prediction':clf.predict(X_test),'Actual':y_test})
    df_prob.index=df_test.index

    best=df_prob.sort_values('Probabilidad').head(titulos)
    worst=df_prob.sort_values('Probabilidad').tail(titulos)
    cartera=best.append(worst)
    
    cartera = cartera.join(df_test)
    res_cartera=cartera[['Prediction','Price_d','Price_d+180']]
    
    res_cartera['Resultado']=res_cartera.apply(resultado,axis=1)*inversion
    
    
    resumen = resumen.append({'Periodo':trim_seleccionado,'Rentabilidad':res_cartera.Resultado.sum()}, ignore_index=True)
    

df_var_indice = pd.read_csv("../tablas/var_indice.csv")
resumen = resumen.merge(df_var_indice, 
                              left_on=('Periodo'),
                              right_on=('Periodo'), 
                              how='left')

resumen.drop(['Indice_d_100','Indice_d+180_100'], axis=1, inplace=True)

resumen
    

,Periodo,Rentabilidad,Rent_indice
0,2016Q3,26.018842,25.447536
1,2016Q4,34.242812,-5.678701
2,2017Q1,41.140106,6.457547
3,2017Q2,27.878416,14.952894
4,2017Q3,-7.561600,9.718820
5,2017Q4,10.460656,4.260848
6,2018Q1,0.150640,-10.306496
7,2018Q2,0.021313,-15.166144
8,2018Q3,31.517821,0.755160
9,2018Q4,6.772955,-10.178698


In [68]:
resumen.Rentabilidad.sum()

170.64196151253154

In [58]:
resumen.Rent_indice.sum()

20.26276498465987

In [59]:
cartera

,Probabilidad,Prediction,Actual,Ticker,Periodo,Price_d,Price_d+180,Etiqueta,PER,Dot_dudosos,...,var_ROA,var_Dot_dudosos,quantile_PER,quantile_PBC,quantile_ROA,quantile_Dot_dudosos,var_quantile_PER,var_quantile_PBC,var_quantile_ROA,var_quantile_Dot_dudosos
931,0.311530,Peor,Peor,WSBC,2018Q4,39.75,32.57,Peor,49.523996,-0.000229,...,-0.268739,-0.641279,73.333333,36.666667,60.000000,80.000000,-11.960784,16.078431,39.411765,-5.294118
85,0.370079,Peor,Peor,CIT,2018Q4,47.55,39.20,Peor,59.964999,-0.000643,...,0.574298,0.203182,90.000000,13.333333,10.000000,20.000000,75.294118,7.450980,-13.529412,11.176471
755,0.377943,Peor,Peor,STBA,2018Q4,39.37,33.61,Peor,50.952110,-0.000375,...,0.173727,-0.826381,83.333333,66.666667,66.666667,56.666667,36.274510,-3.921569,-6.862745,-31.568627
696,0.383145,Peor,Peor,SIVB,2018Q4,242.13,199.23,Peor,48.314331,-0.000240,...,0.010611,0.232487,70.000000,96.666667,93.333333,76.666667,-3.529412,-0.392157,-0.784314,26.666667
725,0.388483,Peor,Mejor,SRCE,2018Q4,45.78,43.27,Mejor,55.236962,-0.000747,...,-0.072563,0.309563,86.666667,76.666667,53.333333,13.333333,-1.568627,-2.745098,6.274510,7.450980
58,0.388830,Peor,Peor,CFR,2018Q4,93.71,82.23,Peor,50.709054,-0.000117,...,0.021825,-0.272425,80.000000,86.666667,63.333333,90.000000,9.411765,1.372549,-4.313725,7.647059
844,0.409552,Peor,Mejor,UMBF,2018Q4,67.18,59.85,Mejor,129.044680,-0.002056,...,1.472692,-0.869666,93.333333,70.000000,3.333333,3.333333,10.980392,8.235294,-23.137255,-58.431373
343,0.583723,Mejor,Mejor,IBKC,2018Q4,75.20,68.12,Mejor,32.307155,-0.000445,...,-0.223457,-0.171204,6.666667,23.333333,70.000000,40.000000,-37.450980,-0.196078,20.000000,1.764706
174,0.585056,Mejor,Mejor,FITB,2018Q4,25.83,24.57,Mejor,37.626204,-0.000650,...,-0.038094,-0.066726,23.333333,26.666667,40.000000,16.666667,-17.843137,0.196078,7.647059,-0.980392
901,0.595080,Mejor,Mejor,WFC,2018Q4,46.78,43.04,Mejor,38.198069,-0.000275,...,-0.033501,0.126856,26.666667,40.000000,36.666667,66.666667,-32.156863,-1.176471,7.254902,19.607843
